# Imports

In [ ]:
import numpy as np
import tensorboard as tb
from tensorboardX import SummaryWriter

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['figure.figsize'] = (15, 5)

In [ ]:
%run ../utils/files.py

In [ ]:
# TB_ID = 'SNN6AQpdQc6ILAVWPZ7LyA'

# Load data

In [ ]:
experiment = tb.data.experimental.ExperimentFromDev(TB_ID)

In [ ]:
df = experiment.get_scalars(include_wall_time=True)
df.head()

In [ ]:
runs = list(df['run'].unique())
len(df), len(runs)

In [ ]:
metrics = list(df['tag'].unique())
len(metrics)

# Add chex-f1 without NF

In [ ]:
for run in runs:
    run_id = RunId(run, debug=False, task='rg', experiment='best-cnn')
    writer = SummaryWriter(get_tb_log_folder(run_id))

    d = df.loc[df['run'] == run]
    d = d.loc[d['tag'].str.contains('chex_f1')]
    d = d.loc[~d['tag'].str.contains('No_Finding')]
    d2 = d.replace(r'chex_f1_\w+/(\w+)', r'\1', regex=True)
    s = d2.groupby(['tag', 'step'])['value'].apply(np.mean)

    for split in ('train', 'val'):
        for step, value in s[split].items():
            writer.add_scalar(f'Chex_f1_woNF/{split}', value, step)
            
    writer.close()

In [ ]:
n_rows = 2
n_cols = 1

print(run)
for i, split in enumerate(('train', 'val')):
    plt.subplot(n_rows, n_cols, i + 1)
    plt.title(split)
    plt.plot(s[split].index, s[split].values)

# Re-write metrics

Fix names

In [ ]:
import re
from tqdm.auto import tqdm
import numpy as np

## Function to rename

###  RG

In [ ]:
TASK = 'rg'

In [ ]:
regex = re.compile(r'chex_(acc|npv|prec|recall|spec)/(train|val)\Z')
def _rename_tag(tag):
    if regex.match(tag):
        return tag.capitalize()
    if 'fonly' in tag:
        tag = tag.replace('fonly', 'woNF')
        tag = tag.replace('chex', 'Chex')
        return tag
    
    return tag

### Cls / cls-seg

In [ ]:
# TASK = 'cls-seg'
TASK = 'cls'

In [ ]:
def _rename_tag(tag):
    tag = tag.replace('pr_auc/', 'PR_auc/')
    tag = tag.replace('recall/', 'Recall/')
    tag = tag.replace('prec/', 'Prec/')
    tag = tag.replace('spec/', 'Spec/')
    return tag

### Check result

In [ ]:
new_metrics = []
for m in metrics:
    m2 = _rename_tag(m)
    if m2 not in metrics:
        new_metrics.append((m, m2))
    print(f'{m:<42} {m2 if m2 != m else "SAME"}')
new_metrics

## Rename

In [ ]:
runs = list(df['run'].unique())
len(runs)

In [ ]:
runs = [
    # '0420_223832_iu-x-ray_lstm-v2_lr0.0001_precnn-0420-174100_normS_size256_front__best-cnn',
    # '0421_062547_iu-x-ray_lstm-att-v2_lr0.0001_precnn-0420-174100_normS_size256_front__best-cnn'
]

In [ ]:
for run in tqdm(runs):
    if run.startswith('fixed'):
        continue

    run_id = RunId(run, debug=False, task=TASK)
    folder = get_tb_log_folder(run_id)
    folder = folder.replace('/runs/', '/runs/fixed/')
    writer = SummaryWriter(folder)

    sub_df = df.loc[df['run'] == run]
    
    if TASK == 'rg':
        # Create a step-to-walltime dict
        d = sub_df.loc[sub_df['tag'] == 'Bleu/train']
        step_to_walltime = dict(zip(d.step, d.wall_time))
    else:
        step_to_walltime = dict()

    for index, row in sub_df.iterrows():
        tag = row['tag']
        step = row['step']
        value = row['value']
        if TASK == 'rg' and step not in step_to_walltime:
            print('Step not in s2w: ', run, step, )
        walltime = step_to_walltime.get(step, row['wall_time'])
        
        if run.startswith('0420_031015') and 'chex_f1_fonly' in tag:
            # This one is repeated
            continue

        tag = _rename_tag(tag)
        writer.add_scalar(tag, value, step, walltime)
    
    writer.close()